# 📁 E-Waste Image Classification using EfficientNetV2B0

In [1]:

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import image_dataset_from_directory
from keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.applications.efficientnet_v2 import preprocess_input
from keras.applications import EfficientNetV2B0
import gradio as gr


In [ ]:

DATASET_PATH = r"D:\E-waste generation classification"

train_dataset = image_dataset_from_directory(
    os.path.join(DATASET_PATH, "train"),
    image_size=(128, 128),
    batch_size=100
)

val_dataset = image_dataset_from_directory(
    os.path.join(DATASET_PATH, "validation"),
    image_size=(128, 128),
    batch_size=100
)

test_dataset = image_dataset_from_directory(
    os.path.join(DATASET_PATH, "test"),
    image_size=(128, 128),
    batch_size=100
)

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y)).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.map(lambda x, y: (preprocess_input(x), y)).cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.map(lambda x, y: (preprocess_input(x), y)).cache().prefetch(buffer_size=AUTOTUNE)


Found 2400 files belonging to 10 classes.
Found 300 files belonging to 10 classes.
Found 300 files belonging to 10 classes.


In [4]:

base_model = EfficientNetV2B0(include_top=False, input_tensor=Input(shape=(128, 128, 3)), weights="imagenet")
base_model.trainable = True

for layer in base_model.layers[:100]:
    layer.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.2)(x)
output = Dense(10, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)


In [8]:

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)


In [9]:

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    callbacks=[early_stop]
)


Epoch 1/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - accuracy: 0.1908 - loss: 2.2652 - val_accuracy: 0.6233 - val_loss: 1.5630
Epoch 2/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.6638 - loss: 1.3980 - val_accuracy: 0.8267 - val_loss: 0.9605
Epoch 3/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.8815 - loss: 0.7697 - val_accuracy: 0.8800 - val_loss: 0.6045
Epoch 4/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.9371 - loss: 0.4636 - val_accuracy: 0.9100 - val_loss: 0.4164
Epoch 5/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.9471 - loss: 0.2872 - val_accuracy: 0.9200 - val_loss: 0.3167
Epoch 6/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.9768 - loss: 0.1792 - val_accuracy: 0.9367 - val_loss: 0.2568
Epoch 7/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.9835 - loss: 0.1273 - val_accuracy: 0.9533 - val_loss: 0.2221
Epoch 8/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.9900 - loss: 0.0953 - val_accuracy: 0.9567 - val_loss:

In [10]:

loss, accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 572ms/step - accuracy: 0.9688 - loss: 0.1139
Test Loss: 0.1446
Test Accuracy: 0.9600


In [12]:

# Save the trained model
model.save("e_waste_classifier_model.keras")


In [ ]:

# Extract class_names from the original dataset object before mapping
class_names = train_dataset.class_names if hasattr(train_dataset, 'class_names') else [
    'Battery', 'Keyboard', 'Microwave', 'Mobile', 'Mouse', 'PCB', 'Player', 'Printer', 'TV', 'Washing Machine'
]

def classify_image(img):
    img = tf.image.resize(img, [128, 128])
    img = preprocess_input(img)
    img = tf.expand_dims(img, axis=0)
    predictions = model.predict(img)
    label = class_names[np.argmax(predictions)]
    confidence = np.max(predictions)
    return {label: float(confidence)}

interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="E-Waste Image Classifier",
    description="Upload an e-waste image to classify into 1 of 10 categories."
)

interface.launch(share=True)  


* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Missing file: C:\Users\pranj\.cache\huggingface\gradio\frpc\frpc_windows_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.3
3. Move the file to this location: C:\Users\pranj\.cache\huggingface\gradio\frpc


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
